# Response table for the GoNoGo task



This code collects the response and response time from each participant's file and save the information in a csv table.

The code first opens each txt file corresponding to the GoNoGo tasks stored in ***path*** . The ID for each participant is collected from the ***Subject*** line in the txt file, and stored in a variable ***ID***.

Parsing of the file is performed by finding the experimental and control samples for each trial (***GoNoGo1, MotorCont1, GoNoGo3, MotorCont2, GoNoGo4, MotorCont3***) within a ***LogFrame***. Two variables are stored for each sample:

- Response time (***RT***): As found in line ***ShowImage.RT***, or 999 if none.
- Response (***Resp***): 1 if the participant answered correrctly (***ShowImage.RESP*** = ***ShowImage.CRESP***), 0 otherwise.

Variables are stored as ***trialVar\_S#\_RT***  and ***trialVar\_S#\_Resp***, with *#* being the number of the sample for each trial, and _trialVar_ the variable for each trial as follow:

***Exp1*** for ***GoNoGo1***, ***Exp2*** for ***GoNoGo3***, ***Exp3*** for ***GoNoGo4***, ***Cont1*** for ***MotorCont1***, ***Cont2*** for ***MotorCont2***, ***Cont3*** for ***MotorCont3***.

For example, in the frame corresponding to trial GoNoGo3, sample 4:

	*** LogFrame Start ***
	Procedure: ExpTrial
	TrialType: Go
	StimPic: sun1.jpg
	CorrectResp: 1
	TriggerValue: 2
	GoNoGo3: 4
	Running: GoNoGo3
	ShowImage.OnsetDelay: 33
	ShowImage.OnsetTime: 202397
	ShowImage.DurationError: 0
	ShowImage.RTTime: 202958
	ShowImage.ACC: 1
	ShowImage.RT: 561
	ShowImage.RESP: 1
	ShowImage.CRESP: 1
	ShowImage.OnsetToOnsetTime: 1533
	GoNoGo3.Cycle: 1
	GoNoGo3.Sample: 4
	*** LogFrame End ***
    
The value of 561 is stored in *Exp2\_S4\_RT* and 1 is stored in *Exp2\_S4\_Resp*

Variables are stored in a csv file. There are 241 variables total in the csv file: 20 for each trial plus the participant's ID.

In [2]:
import pandas as pd
import os, sys


In [3]:

path = "/media/ioastorga/Lexar/NIRS_data"
task_files_all = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
        if ".txt" in file and "GoNoGo" in file and not "Practice" in file and not "._" in file:
            task_files_all.append(os.path.join(r,file))
print(task_files_all)

['/media/ioastorga/Lexar/NIRS_data/10040_20181215/0fNIRS GoNoGo Task Touchsreeen_JM_Triggers-10040-1.txt', '/media/ioastorga/Lexar/NIRS_data/10046_20190304/0fNIRS GoNoGo Task Touchsreeen_JM_Triggers-10046-1.txt', '/media/ioastorga/Lexar/NIRS_data/10051_20190425/0fNIRS GoNoGo Task Touchsreeen_JM_Triggers_StimBlack-10051-1.txt', '/media/ioastorga/Lexar/NIRS_data/10537_20190202/0fNIRS GoNoGo Task Touchsreeen_JM_Triggers-10537-1.txt', '/media/ioastorga/Lexar/NIRS_data/10538_20181208/0fNIRS GoNoGo Task Touchsreeen-10538-1.txt', '/media/ioastorga/Lexar/NIRS_data/10560_20190615/0fNIRS GoNoGo Task Touchsreeen_JM_Triggers_StimBlack-10560-1.txt', '/media/ioastorga/Lexar/NIRS_data/10609_20190223/0fNIRS GoNoGo Task Touchsreeen_JM_Triggers-10609-1.txt', '/media/ioastorga/Lexar/NIRS_data/10614_20190622/0fNIRS GoNoGo Task Touchsreeen_JM_Triggers_StimBlack-10614-1.txt', '/media/ioastorga/Lexar/NIRS_data/10616_20190124/0fNIRS GoNoGo Task Touchsreeen_JM_Triggers-10616-1.txt', '/media/ioastorga/Lexar/NIR

path = "10040_20181215"
dir_to_list = os.listdir(path)
gonogo_files_all = []
for item in dir_to_list:
    if ".txt" in item and "GoNoGo" in item and not "Practice" in item and not "._" in item:
        gonogo_files_all.append(path+'/'+item)
print(gonogo_files_all

path = "gonogo_dir1"
dir_to_list = os.listdir(path)
gonogo_files = []
for item in dir_to_list:
    if ".txt" in item:
        gonogo_files.append(path+'/'+item)
print(gonogo_files)

In [7]:
field_to_column = dict([('GoNoGo1','Exp1'), ('MotorCont1','Ctr1'),
                   ('GoNoGo3','Exp3'), ('MotorCont2','Ctr2'),
                   ('GoNoGo4','Exp4'), ('MotorCont3','Ctr3'),]) 


def file_to_row(fname):
    row = {}
    start_parsing_frame = False
    curr_frame = {}
    with open(fname, encoding='utf-16-le') as fl:
        for line in fl.readlines():
            line = line.strip('\n')
            if 'Subject:' in line:
                row['ID'] = line.split(':')[-1].strip()
            elif 'LogFrame Start' in line:
                start_parsing_frame = True
            elif 'LogFrame End' in line:
                start_parsing_frame = False
                frame_type = None
                frame_key = None
                for key in curr_frame.keys():
                    for field_key in field_to_column.keys():
                        if key == field_key:
                            frame_type = field_to_column[key] # e.g. Exp1
                            frame_key = key
                if frame_type is None:
                    curr_frame = {}
                else:
                    sample = curr_frame[frame_key] # S: 1-20
                    column_name = frame_type + '_S' + sample
                    if 'ShowImage.RT' in curr_frame:
                        row[column_name + '_RT'] = int(curr_frame['ShowImage.RT'])
                    else:
                        row[column_name + '_RT'] = 999 # or None? check
                    if 'ShowImage.RESP' in curr_frame and curr_frame['ShowImage.RESP'] == curr_frame['ShowImage.CRESP']:
                        row[column_name + '_Resp'] = 1
                    else:
                        row[column_name + '_Resp'] = 0 # or None? check

                    curr_frame = {}
            elif start_parsing_frame:
                fields = [x.strip() for x in line.split(':')]
                curr_frame[fields[0]] = fields[1]
    return row
            
rows = []
# for fname in ['0fNIRS GoNoGo Task Touchsreeen_JM_Triggers_Stim1-10503-1.txt']:
#for fname in gonogo_files:
for fname in task_files_all:
    rows.append(file_to_row(fname))

expt_table = pd.DataFrame(rows).set_index('ID')



In [8]:
expt_table.to_csv('table_for_spss_all.csv')

In [9]:
len(rows)

31